#### Training GCN

Model using the graph dataset extracted from the csv file. TO generate csv file, please run the parsing notebook. 

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import dgl
import dgl.nn as dglnn
import random
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# ------------------------------
# 1. Load Dataset and Prepare Data
# ------------------------------
df = pd.read_csv("all_circuits_features.csv")
print("\n📌 Dataset Overview:")
print(df.info())

gate_types = ['and', 'or', 'nand', 'nor', 'xor', 'xnor', 'buf', 'not']

# Convert 'gate_type' to numerical labels
label_encoder = LabelEncoder()
df["gate_label"] = label_encoder.fit_transform(df["gate_type"])

# Feature columns
feature_columns = [
    "fan_in", "fan_out", "dist_to_output", "is_primary_input", "is_primary_output",
    "is_internal", "is_key_gate", "degree_centrality", "betweenness_centrality",
    "closeness_centrality", "clustering_coefficient", "avg_fan_in_neighbors", "avg_fan_out_neighbors"
]

# Drop NaN values and convert features to float
df = df.dropna(subset=feature_columns)
df[feature_columns] = df[feature_columns].astype(float)

# Normalize numerical features
scaler = StandardScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])

# ---------------------------
# 2. Build Node and Edge Lists
# ---------------------------
nodes = df["node"].tolist()
node_to_id = {node: i for i, node in enumerate(nodes)}

# Build edge list based on fan-in relationships (directed: from source to current node)
edges = []
for idx, row in df.iterrows():
    node_id = node_to_id[row["node"]]
    # Select potential driving nodes: those with fan_out > 0
    potential_sources = df[df["fan_out"] > 0]["node"].tolist()
    num_fan_in = int(row["fan_in"])
    if num_fan_in > 0:
        sources = potential_sources[:num_fan_in]
        for src in sources:
            if src in node_to_id:
                edges.append((node_to_id[src], node_id))

print("\n🔍 Extracted", len(edges), "edges.")
if len(edges) == 0:
    raise ValueError("No edges found! Check your fan_in values.")

# Convert edges into tensors
src_nodes, dst_nodes = zip(*edges) if edges else ([], [])
src_tensor = torch.tensor(src_nodes, dtype=torch.int64)
dst_tensor = torch.tensor(dst_nodes, dtype=torch.int64)
valid_edges = (src_tensor >= 0) & (dst_tensor >= 0) & (src_tensor < len(nodes)) & (dst_tensor < len(nodes))
src_tensor = src_tensor[valid_edges]
dst_tensor = dst_tensor[valid_edges]

# ---------------------------
# 3. Create the Original DGL Graph
# ---------------------------
graph = dgl.graph((src_tensor, dst_tensor), num_nodes=len(nodes))
graph = dgl.add_self_loop(graph)  # add self-loops to avoid zero in-degree issues

# Assign node features and labels
graph.ndata['features'] = torch.tensor(df[feature_columns].values, dtype=torch.float32)
graph.ndata['labels'] = torch.tensor(df["gate_label"].values, dtype=torch.long)

# Create train-test split using node indices
nodes_idx = np.arange(len(nodes))
train_idx, test_idx = train_test_split(nodes_idx, test_size=0.2, random_state=42)
train_mask = torch.tensor(train_idx, dtype=torch.long)
test_mask = torch.tensor(test_idx, dtype=torch.long)

# ---------------------------
# 4. Define the GCNN Model
# ---------------------------
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GCN, self).__init__()
        self.conv1 = dglnn.GraphConv(in_feats, hidden_feats, allow_zero_in_degree=True)
        self.conv2 = dglnn.GraphConv(hidden_feats, out_feats, allow_zero_in_degree=True)

    def forward(self, g, inputs):
        x = self.conv1(g, inputs)
        x = torch.relu(x)
        x = self.conv2(g, x)
        return x

in_feats = len(feature_columns)
hidden_feats = 32
out_feats = len(label_encoder.classes_)
model = GCN(in_feats, hidden_feats, out_feats)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

# ---------------------------
# 5. Training Loop
# ---------------------------
epochs = 50
for epoch in range(epochs):
    model.train()
    logits = model(graph, graph.ndata['features'])
    loss = loss_fn(logits[train_mask], graph.ndata['labels'][train_mask])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")

# ---------------------------
# 6. Evaluation
# ---------------------------
model.eval()
with torch.no_grad():
    logits = model(graph, graph.ndata['features'])
    test_logits = logits[test_mask]
    test_predictions = test_logits.argmax(dim=1)
    orig_accuracy = (test_predictions == graph.ndata['labels'][test_mask]).float().mean().item()
    
    print(f"\n✅ Test Accuracy on Original Graph: {orig_accuracy * 100:.2f}%")
    
    true_labels_orig = graph.ndata['labels'][test_mask].cpu().numpy()
    pred_labels_orig = test_predictions.cpu().numpy()
    conf_mat_orig = confusion_matrix(true_labels_orig, pred_labels_orig)
    
    print("\n📝 Confusion Matrix (Original Graph):")
    print(conf_mat_orig)
    
    print("\nClassification Report (Original Graph):")
    print(classification_report(true_labels_orig, pred_labels_orig, target_names=label_encoder.classes_))
   


📌 Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60882 entries, 0 to 60881
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   circuit_name            60882 non-null  object 
 1   node                    60882 non-null  object 
 2   gate_type               60882 non-null  object 
 3   fan_in                  60882 non-null  int64  
 4   fan_out                 60882 non-null  int64  
 5   depth                   60882 non-null  object 
 6   dist_to_output          60882 non-null  int64  
 7   is_primary_input        60882 non-null  int64  
 8   is_primary_output       60882 non-null  int64  
 9   is_internal             60882 non-null  int64  
 10  is_key_gate             60882 non-null  int64  
 11  key_dependency          122 non-null    object 
 12  degree_centrality       60882 non-null  float64
 13  betweenness_centrality  60882 non-null  float64
 14  closeness_central

C:\Users\rrk307\Anaconda3\lib\site-packages\dgl\backend\pytorch\tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


Epoch 0/50, Loss: 2.4010
Epoch 10/50, Loss: 1.1221
Epoch 20/50, Loss: 0.5066
Epoch 30/50, Loss: 0.3615
Epoch 40/50, Loss: 0.2943

✅ Test Accuracy on Original Graph: 92.31%

📝 Confusion Matrix (Original Graph):
[[5555    0   29   53    0    0    0    0]
 [   0  472    3    0    0    0    0    0]
 [ 439    0   31   57    0    0    0    0]
 [ 151    0    4  412    0    0    0    0]
 [   0    0    0    4 4474    0    0    0]
 [ 195    0    2    0    0    0    0    0]
 [   0    0    0    0    0    0  266    0]
 [   0    0    0    0    0    0    0   30]]

Classification Report (Original Graph):
              precision    recall  f1-score   support

         and       0.88      0.99      0.93      5637
       input       1.00      0.99      1.00       475
        nand       0.45      0.06      0.10       527
         nor       0.78      0.73      0.75       567
         not       1.00      1.00      1.00      4478
          or       0.00      0.00      0.00       197
      output       1.00  

C:\Users\rrk307\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\rrk307\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\rrk307\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Jacobian 
Calculation for one sample.

In [6]:
#########################
# 7. Jacobian Computation and Verification
#########################

print("\n--- Jacobian Computation and Verification ---\n")

# Select one test node. Here, we simply choose the first node in the test set.
test_idx = test_mask[0].item()
print(f"Selected test node index: {test_idx}")

# Get the feature vector for this test node and set it to require gradients.
x0 = graph.ndata['features'][test_idx].clone().detach().requires_grad_(True)
print("Original feature vector for test node (x0):", x0)

# Define a helper function that takes a feature vector for test_idx and returns the model's output (logit vector)
def f(x):
    """
    Given an input feature vector x for the selected test node (test_idx),
    create a new features tensor by replacing the test node's features and compute the model's output.
    """
    # Clone the original features from the graph.
    new_features = graph.ndata['features'].clone().detach()
    # Replace the test node's feature vector with x.
    new_features[test_idx] = x
    # Forward propagate through the model (the graph structure remains unchanged).
    output = model(graph, new_features)
    # Return the output (logit vector) for the test node.
    return output[test_idx]

# Compute the Jacobian of f with respect to x0.
# The resulting Jacobian has shape (num_classes, feature_dim)
jacobian = torch.autograd.functional.jacobian(f, x0)

print("\nJacobian shape:", jacobian.shape)
print("Jacobian matrix:\n", jacobian)

#########################
# 8. Jacobian Verification via Finite Differences
#########################

# Choose a small random perturbation delta.
epsilon = 1e-3
delta = epsilon * torch.randn_like(x0)
print("\nPerturbation delta vector:", delta)

# Compute the predicted change in output using the linear approximation: J * delta.
predicted_change = jacobian.mv(delta)  # matrix-vector product
print("\nPredicted change in output (Jacobian * delta):", predicted_change)

# Now compute the actual change in output by explicitly perturbing x0.
f_x0 = f(x0)
f_x0_perturbed = f(x0 + delta)
actual_change = f_x0_perturbed - f_x0
print("Actual change in output after perturbation:", actual_change)

# Calculate the relative error between the predicted change and the actual change.
relative_error = (predicted_change - actual_change).norm() / (actual_change.norm() + 1e-8)
print("\nRelative error between predicted and actual change: {:.4e}".format(relative_error.item()))

#########################
# 9. Outcome Interpretation
#########################

if relative_error < 1e-2:
    print("\nOutcome: The Jacobian accurately captures the local sensitivity of the model's output relative to small perturbations in the input.")
else:
    print("\nOutcome: The relatively high error indicates that higher-order effects or estimation noise may be present, suggesting that the local model behavior is less linear.")

#############################################
# 10. Discussion: Verifying Jacobian Information
#############################################
#
# The procedure above gives us two key pieces of information:
#
# 1. The Jacobian matrix tells us how significantly each output logit is affected by changes in each feature component.
#    - For example, large entries in the Jacobian indicate that a small change in that particular input dimension produces a large change in the output.
#
# 2. By comparing the linear prediction (via the Jacobian) to the actual output change for a small input alteration,
#    we verify (via low relative error) that the Jacobian is a valid local approximation.
#
# In the context of circuit analysis:
# - High local sensitivity (i.e. large norms of the Jacobian) might indicate that the GCNN's decision function is "sharp" or brittle in that region.
# - An adversary could use such information to target the most sensitive input directions, while a defender might regularize the model to minimize the Jacobian norm.
#
# Verification:
# - The relative error is used as a measure: If it is low (e.g., < 1%), then the linear approximation (Jacobian) is a reliable indicator of local sensitivity.
# - This provides confidence that the model's predictions are indeed highly sensitive to specific segments of the input space.



--- Jacobian Computation and Verification ---

Selected test node index: 54063
Original feature vector for test node (x0): tensor([ 0.5852,  3.3096, -0.2307, -0.2018, -0.1490,  0.2546, -0.0448,  0.2462,
         0.4904, -0.4170, -0.2078,  0.2185,  0.1667], requires_grad=True)

Jacobian shape: torch.Size([8, 13])
Jacobian matrix:
 tensor([[ 0.8985,  0.3651, -0.3604,  0.2352,  0.5114, -0.2491,  0.1276, -0.2225,
          0.4148,  0.1603, -0.2108,  0.0024, -0.2186],
        [ 0.3432, -0.1366,  0.0459,  0.7290, -0.3247, -0.6114, -0.1305,  0.1473,
         -0.2415, -0.3517,  0.5398,  0.3522,  0.6216],
        [ 0.0298, -0.0806, -0.0050,  0.3152, -0.0629, -0.0671, -0.1677,  0.1670,
         -0.2373,  0.2443,  0.1969,  0.3316,  0.0488],
        [ 0.6087, -0.2459,  0.0078,  0.3005, -0.0187, -0.4585, -0.0212, -0.1625,
          0.0031, -0.5966,  0.3872,  0.1420,  0.5246],
        [-1.6733,  0.0235,  0.0924, -0.9347, -0.5967,  0.7131, -0.6982,  0.0328,
         -0.3167,  0.2452, -0.5859,  0.023

##### Several Samples per class

In [10]:
import torch
import numpy as np

# Define gate types corresponding to the labels
gate_types = ['and', 'or', 'nand', 'nor', 'xor', 'xnor', 'buf', 'not']

# Dictionary to store metrics for each class
class_metrics = {gt: {'jacobian_norms': [], 'relative_errors': []} for gt in gate_types}
overall_jacobian_norms = []
overall_relative_errors = []

# Function to compute the Jacobian for a given test index.
def compute_jacobian_for_sample(test_idx):
    # Get the feature vector for the selected test node and require gradients.
    x0 = graph.ndata['features'][test_idx].clone().detach().requires_grad_(True)
    def f(x):
        new_features = graph.ndata['features'].clone().detach()
        new_features[test_idx] = x
        output = model(graph, new_features)
        return output[test_idx]
    # Compute the Jacobian of f with respect to x0.
    jacobian = torch.autograd.functional.jacobian(f, x0)
    return jacobian, x0, f

# Loop over each selected test node (100 per class)
# Taken same as earlier metric (Local Lipschitz Constant) copy that code here.
for test_idx in selected_test_nodes:
    # Determine the ground-truth label and corresponding gate type.
    label_idx = graph.ndata['labels'][test_idx].item()
    class_name = gate_types[label_idx]
    
    # Compute the Jacobian for the selected test node.
    jacobian, x0, f = compute_jacobian_for_sample(test_idx)
    # Compute the Frobenius norm of the Jacobian.
    jacobian_norm = torch.norm(jacobian, p='fro').item()
    
    # Finite-difference verification:
    epsilon = 1e-3
    delta = epsilon * torch.randn_like(x0)
    predicted_change = jacobian.mv(delta)
    f_x0 = f(x0)
    f_x0_perturbed = f(x0 + delta)
    actual_change = f_x0_perturbed - f_x0
    relative_error = (torch.norm(predicted_change - actual_change) /
                      (torch.norm(actual_change) + 1e-8)).item()
    
    # Store values for the current sample.
    class_metrics[class_name]['jacobian_norms'].append(jacobian_norm)
    class_metrics[class_name]['relative_errors'].append(relative_error)
    overall_jacobian_norms.append(jacobian_norm)
    overall_relative_errors.append(relative_error)

# Display per-class results in a tabular format.
print("\nClass-wise Jacobian Analysis Results:")
header = "{:<10s} {:>30s} {:>35s}".format("Class", "Avg. Jacobian Norm ± Std", "Avg. Relative Error ± Std")
print(header)
print("-" * len(header))

for cls in gate_types:
    norms = class_metrics[cls]['jacobian_norms']
    errors = class_metrics[cls]['relative_errors']
    avg_norm = np.mean(norms) if norms else 0
    std_norm = np.std(norms) if norms else 0
    avg_rel_error = np.mean(errors) if errors else 0
    std_rel_error = np.std(errors) if errors else 0
    print("{:<10s} {:>15.4f} ± {:<12.4f} {:>15.4e} ± {:<10.4e}".format(
        cls, avg_norm, std_norm, avg_rel_error, std_rel_error))

# Compute overall aggregated metrics.
overall_avg_norm = np.mean(overall_jacobian_norms)
overall_std_norm = np.std(overall_jacobian_norms)
overall_avg_rel_error = np.mean(overall_relative_errors)
overall_std_rel_error = np.std(overall_relative_errors)

print("\nOverall Aggregated Jacobian Analysis Results:")
print("Average Jacobian Norm: {:.4f} ± {:.4f}".format(overall_avg_norm, overall_std_norm))
print("Average Relative Error: {:.4e} ± {:.4e}".format(overall_avg_rel_error, overall_std_rel_error))



Class-wise Jacobian Analysis Results:
Class            Avg. Jacobian Norm ± Std           Avg. Relative Error ± Std
-----------------------------------------------------------------------------
and                 7.4365 ± 1.5834            9.8170e-04 ± 7.2927e-03
or                  5.7241 ± 0.4275            2.9550e-04 ± 1.7438e-04
nand                6.4330 ± 1.9736            1.4521e-04 ± 1.7947e-04
nor                 5.5897 ± 0.9371            1.7155e-04 ± 1.1367e-04
xor                 8.6396 ± 1.1227            1.2890e-04 ± 7.7688e-05
xnor                6.0443 ± 2.4619            1.2810e-02 ± 4.6570e-02
buf                 4.5112 ± 0.1996            4.3216e-04 ± 2.4847e-04
not                 4.0101 ± 0.0797            1.0110e-03 ± 4.8172e-04

Overall Aggregated Jacobian Analysis Results:
Average Jacobian Norm: 6.2440 ± 1.9325
Average Relative Error: 2.0916e-03 ± 1.7965e-02


In [24]:
import numpy as np
import torch
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support

# Ensure we have a NumPy array of node indices for the test set
if torch.is_tensor(test_mask):
    test_indices = test_mask.cpu().numpy()
else:
    test_indices = np.array(test_mask)

labels_np = graph.ndata['labels'].cpu().numpy()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)

rng = np.random.default_rng(seed=42)
PER_CLASS = 100
selected_test_nodes = []

print("\nSampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)

    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set — skipped.")
        continue

    if n_avail >= PER_CLASS:
        chosen = rng.choice(idxs, size=PER_CLASS, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = idxs
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")

    selected_test_nodes.extend(chosen)

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

# Sanity check counts
counts = {class_names[c]: int(np.sum(labels_np[selected_test_nodes] == c)) for c in range(num_classes)}
print("\nFinal counts in selected perturbed set:")
for name, cnt in counts.items():
    print(f"- {name}: {cnt}")

# Clone features for perturbation
perturbed_features = graph.ndata['features'].clone().detach()
epsilon = 0.05  # tweak for stronger/weaker perturbations

for node_idx in selected_test_nodes:
    x0 = graph.ndata['features'][node_idx]
    if 'jacobian_perturbations' in globals() and node_idx in jacobian_perturbations:
        delta = jacobian_perturbations[node_idx]
        norm = torch.norm(delta)
        if norm.item() > 0:
            delta = epsilon * delta / norm
    else:
        delta = epsilon * torch.randn_like(x0)
    perturbed_features[node_idx] = (x0 + delta)

eval_idx = torch.tensor(selected_test_nodes, dtype=torch.long)
model.eval()
with torch.no_grad():
    logits_pert = model(graph, perturbed_features)
    preds = logits_pert[eval_idx].argmax(dim=1).cpu().numpy()
    labels_true = labels_np[eval_idx]

acc = np.mean(preds == labels_true)
prec, rec, f1, _ = precision_recall_fscore_support(labels_true, preds, average='weighted', zero_division=0)

print(f"\n✅ Accuracy on Perturbed Set: {acc * 100:.2f}%")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")

cm = confusion_matrix(labels_true, preds, labels=np.arange(num_classes))
print("\nConfusion Matrix (Perturbed Set, all classes):")
print(cm)

present_labels = np.unique(labels_true)
present_names = [class_names[i] for i in present_labels]
print("\nClassification Report (present classes only):")
print(classification_report(labels_true, preds,
                            labels=present_labels,
                            target_names=present_names,
                            zero_division=0))
    


Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

Final counts in selected perturbed set:
- and: 100
- input: 100
- nand: 100
- nor: 100
- not: 100
- or: 100
- output: 100
- xor: 30

✅ Accuracy on Perturbed Set: 69.32%
Precision: 0.6781
Recall:    0.6932
F1 Score:  0.6338

Confusion Matrix (Perturbed Set, all classes):
[[ 99   0   0   1   0   0   0   0]
 [  0  99   1   0   0   0   0   0]
 [ 82   0   3  15   0   0   0   0]
 [ 24   0   1  75   0   0   0   0]
 [  0   0   0   0 100   0   0   0]
 [ 99   0   1   0   0   0   0   0]
 [  0   0   0   0   0   0 100   0]
 [  0   0   0   0   0   0   0  30]]

Classification Report (present classes only):
              precision    recall  f1-score   support

         and    

#### Local Lipschitz constant

In [9]:
import torch
import numpy as np

print("\n--- Local Lipschitz Constant Computation and Verification (100 Samples Per Class) ---\n")

# Initialize dictionaries to store the local Lipschitz constants (L_i) for each class.
class_lipschitz = {gt: [] for gt in gate_types}
overall_lipschitz = []

def compute_jacobian_for_sample(test_idx):
    """
    Compute the Jacobian matrix for a given test index.
    Returns the Jacobian, the original input x0, and the output function f.
    """
    x0 = graph.ndata['features'][test_idx].clone().detach().requires_grad_(True)
    
    def f(x):
        new_features = graph.ndata['features'].clone().detach()
        new_features[test_idx] = x
        output = model(graph, new_features)
        return output[test_idx]
    
    jacobian = torch.autograd.functional.jacobian(f, x0)
    return jacobian, x0, f

# Iterate over each selected test node from the chosen 100 samples per class.
for test_idx in selected_test_nodes:
    label_idx = graph.ndata['labels'][test_idx].item()
    class_name = gate_types[label_idx]
    
    # Compute the Jacobian for this sample.
    jacobian, x0, f = compute_jacobian_for_sample(test_idx)
    
    # The local Lipschitz constant is approximated as the spectral norm (i.e., the largest singular value)
    # of the Jacobian matrix.
    L_local = torch.linalg.norm(jacobian, ord=2).item()
    
    # Store the value per class and overall.
    class_lipschitz[class_name].append(L_local)
    overall_lipschitz.append(L_local)

# Print the per-class results in a tabular format.
print("\nClass-wise Local Lipschitz Constant Results:")
header = "{:<10s} {:>30s}".format("Class", "Avg Lipschitz Constant ± Std")
print(header)
print("-" * len(header))

for cls in gate_types:
    values = class_lipschitz[cls]
    if len(values) > 0:
        avg_val = np.mean(values)
        std_val = np.std(values)
    else:
        avg_val = 0
        std_val = 0
    print("{:<10s} {:>15.4f} ± {:<12.4f}".format(cls, avg_val, std_val))
    
# Compute and display the overall aggregated metrics.
overall_avg = np.mean(overall_lipschitz)
overall_std = np.std(overall_lipschitz)
print("\nOverall Aggregated Local Lipschitz Constant:")
print("Average Lipschitz Constant: {:.4f} ± {:.4f}".format(overall_avg, overall_std))



--- Local Lipschitz Constant Computation and Verification (100 Samples Per Class) ---


Class-wise Local Lipschitz Constant Results:
Class        Avg Lipschitz Constant ± Std
-----------------------------------------
and                 5.5017 ± 1.1898      
or                  5.1081 ± 0.4761      
nand                4.6870 ± 1.4592      
nor                 4.2143 ± 0.6937      
xor                 7.0457 ± 0.9065      
xnor                4.4031 ± 1.7521      
buf                 3.2364 ± 0.1262      
not                 2.9905 ± 0.0312      

Overall Aggregated Local Lipschitz Constant:
Average Lipschitz Constant: 4.8073 ± 1.5585


In [8]:
import torch
import numpy as np

print("\n--- Local Lipschitz Constant Computation & Relative Error Verification (100 Samples Per Class) ---\n")

# ✅ Get class names directly from label_encoder so they match the label indices
class_names = list(label_encoder.classes_)

# -----------------------------
# 1. Build selected_test_nodes
# -----------------------------
rng = np.random.default_rng(42)  # for reproducibility
test_labels = graph.ndata['labels'][test_mask].cpu().numpy()
test_indices = test_mask.cpu().numpy()

# Group test indices by class
per_class_indices = {cls_idx: [] for cls_idx in range(len(class_names))}
for idx, lbl in zip(test_indices, test_labels):
    per_class_indices[lbl].append(idx)

# Randomly choose up to 100 samples per class
selected_test_nodes = []
for cls_idx, idxs in per_class_indices.items():
    if len(idxs) > 0:
        k = min(100, len(idxs))
        chosen = rng.choice(idxs, size=k, replace=False).tolist()
        selected_test_nodes.extend(chosen)

# -----------------------------
# 2. Metrics storage
# -----------------------------
class_lipschitz = {cn: [] for cn in class_names}
class_rel_errors = {cn: [] for cn in class_names}
overall_lipschitz = []
overall_rel_errors = []

# -----------------------------
# 3. Jacobian helper
# -----------------------------
def compute_jacobian_for_sample(test_idx):
    x0 = graph.ndata['features'][test_idx].clone().detach().requires_grad_(True)
    def f(x):
        new_features = graph.ndata['features'].clone().detach()
        new_features[test_idx] = x
        return model(graph, new_features)[test_idx]
    J = torch.autograd.functional.jacobian(f, x0)
    return J, x0, f

# -----------------------------
# 4. Main loop with relative error check
# -----------------------------
epsilon = 1e-3
trials_per_node = 10

for test_idx in selected_test_nodes:
    label_idx = graph.ndata['labels'][test_idx].item()
    class_name = class_names[label_idx]

    J, x0, f = compute_jacobian_for_sample(test_idx)
    L_local = torch.linalg.norm(J, ord=2).item()

    # Relative error: average over multiple random perturbations
    rel_errors_for_node = []
    for _ in range(trials_per_node):
        delta = epsilon * torch.randn_like(x0)
        pred_change_norm = L_local * torch.norm(delta).item()
        actual_change_norm = torch.norm(f(x0 + delta) - f(x0)).item()
        rel_error = abs(pred_change_norm - actual_change_norm) / (actual_change_norm + 1e-8)
        rel_errors_for_node.append(rel_error)

    avg_rel_error_node = float(np.mean(rel_errors_for_node))

    # Store
    class_lipschitz[class_name].append(L_local)
    class_rel_errors[class_name].append(avg_rel_error_node)
    overall_lipschitz.append(L_local)
    overall_rel_errors.append(avg_rel_error_node)

# -----------------------------
# 5. Reporting
# -----------------------------
print("\nClass-wise Local Lipschitz (with Relative Errors):")
header = "{:<12s} {:>25s} {:>30s}".format("Class", "Avg Lipschitz ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    L_vals = class_lipschitz[cn]
    E_vals = class_rel_errors[cn]
    if L_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(L_vals), np.std(L_vals), np.mean(E_vals), np.std(E_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:>10s} {:>15s} {:>10s}".format(cn, "-", "-", "-", "-"))

print("\nOverall Aggregated:")
print("Avg Lipschitz Constant: {:.4f} ± {:.4f}".format(np.mean(overall_lipschitz), np.std(overall_lipschitz)))
print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))



--- Local Lipschitz Constant Computation & Relative Error Verification (100 Samples Per Class) ---


Class-wise Local Lipschitz (with Relative Errors):
Class              Avg Lipschitz ± Std           Avg Rel. Error ± Std
---------------------------------------------------------------------
and               5.5017 ± 1.1898          2.4428e+00 ± 6.0427e-01
input             5.1081 ± 0.4761          3.5274e+00 ± 8.0134e-01
nand              4.6870 ± 1.4592          2.3030e+00 ± 5.6106e-01
nor               4.2143 ± 0.6937          2.3866e+00 ± 4.9467e-01
not               7.0457 ± 0.9065          2.9362e+00 ± 7.1395e-01
or                4.4031 ± 1.7521          2.3817e+00 ± 5.2646e-01
output            3.2364 ± 0.1262          2.1944e+00 ± 4.3944e-01
xor               2.9905 ± 0.0312          2.3049e+00 ± 3.0851e-01

Overall Aggregated:
Avg Lipschitz Constant: 4.8073 ± 1.5585
Avg Relative Error: 2.5840e+00 ± 7.3497e-01


In [30]:
import numpy as np
import torch
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support

# -------------------------------------------------
# Assume:
#  - `selected_test_nodes` already exists (from sampling step)
#  - `jacobian_perturbations` is a dict: node_idx -> perturbation tensor
#  - `graph`, `model`, `label_encoder` already defined
# -------------------------------------------------

# Ensure we have a NumPy array of node indices for the test set
if torch.is_tensor(test_mask):
    test_indices = test_mask.cpu().numpy()
else:
    test_indices = np.array(test_mask)

labels_np = graph.ndata['labels'].cpu().numpy()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)

rng = np.random.default_rng(seed=12)
PER_CLASS = 100
selected_test_nodes = []

print("\nSampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)

    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set — skipped.")
        continue

    if n_avail >= PER_CLASS:
        chosen = rng.choice(idxs, size=PER_CLASS, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = idxs
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")

    selected_test_nodes.extend(chosen)

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

labels_np = graph.ndata['labels'].cpu().numpy()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)

# Clone features for perturbation
perturbed_features = graph.ndata['features'].clone().detach()

epsilon = 0.05  # adjust perturbation strength if needed

for node_idx in selected_test_nodes:
    x0 = graph.ndata['features'][node_idx]
    if 'jacobian_perturbations' in globals() and node_idx in jacobian_perturbations:
        delta = jacobian_perturbations[node_idx]
        norm = torch.norm(delta)
        if norm.item() > 0:
            delta = epsilon * delta / norm
    else:
        delta = epsilon * torch.randn_like(x0)
    perturbed_features[node_idx] = (x0 + delta)

# Evaluate only on these perturbed nodes
eval_idx = torch.tensor(selected_test_nodes, dtype=torch.long)
model.eval()
with torch.no_grad():
    logits_pert = model(graph, perturbed_features)
    preds = logits_pert[eval_idx].argmax(dim=1).cpu().numpy()
    labels_true = labels_np[eval_idx]

# Metrics
acc = np.mean(preds == labels_true)
prec, rec, f1, _ = precision_recall_fscore_support(labels_true, preds, average='weighted', zero_division=0)

print(f"\n✅ Accuracy on Perturbed Set: {acc * 100:.2f}%")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")

# Confusion matrix for all classes
cm = confusion_matrix(labels_true, preds, labels=np.arange(num_classes))
print("\nConfusion Matrix (Perturbed Set, all classes):")
print(cm)

# Classification report for present classes only
present_labels = np.unique(labels_true)
present_names = [class_names[i] for i in present_labels]
print("\nClassification Report (present classes only):")
print(classification_report(labels_true, preds,
                            labels=present_labels,
                            target_names=present_names,
                            zero_division=0))



Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

✅ Accuracy on Perturbed Set: 69.59%
Precision: 0.7171
Recall:    0.6959
F1 Score:  0.6479

Confusion Matrix (Perturbed Set, all classes):
[[ 96   0   1   3   0   0   0   0]
 [  0 100   0   0   0   0   0   0]
 [ 83   0   9   8   0   0   0   0]
 [ 26   0   1  73   0   0   0   0]
 [  0   0   0   0 100   0   0   0]
 [ 99   0   1   0   0   0   0   0]
 [  0   0   0   0   0   0 100   0]
 [  0   0   0   0   0   0   0  30]]

Classification Report (present classes only):
              precision    recall  f1-score   support

         and       0.32      0.96      0.48       100
       input       1.00      1.00      1.00       100
        nand       0.75      0.09      0.

#### Hessian-Based Curvature Measure

In [11]:
import torch
import numpy as np

print("\n--- Hessian-Based Curvature Measure Computation (100 Samples Per Class) ---\n")

# Define gate types corresponding to your labels.
gate_types = ['and', 'or', 'nand', 'nor', 'xor', 'xnor', 'buf', 'not']

# Dictionary to store the maximum eigenvalue (lambda_max) for each gate type.
class_hessian_eig = {gt: [] for gt in gate_types}
overall_hessian_eig = []

def compute_hessian_for_sample(test_idx):
    """
    Compute the Hessian-based curvature measure for a given test sample.
    We define a scalar function h(x) as the log probability (via log-softmax) 
    of the predicted class (determined on the unperturbed input). This helps
    introduce nonlinearity, thereby yielding nonzero second-order derivatives.
    """
    # Retrieve the feature vector for the test node and require gradients.
    x0 = graph.ndata['features'][test_idx].clone().detach().requires_grad_(True)
    
    # Determine the predicted class for the test node.
    with torch.no_grad():
        logits = model(graph, graph.ndata['features'])
        pred_class = torch.argmax(logits[test_idx])
    
    # Define a scalar function h(x) that returns log probability for the predicted class.
    def h(x):
        new_features = graph.ndata['features'].clone().detach()
        new_features[test_idx] = x
        logits = model(graph, new_features)[test_idx]
        log_probs = torch.log_softmax(logits, dim=0)
        return log_probs[pred_class]
    
    # Compute the Hessian of h(x) at x0.
    H = torch.autograd.functional.hessian(h, x0, create_graph=False, vectorize=False)
    H = H.detach()
    
    # Compute eigenvalues using a symmetric eigendecomposition.
    eigvals = torch.linalg.eigvalsh(H)
    lambda_max = torch.max(eigvals).item()
    return lambda_max, x0, h

# Iterate over each selected test node (assumed to be 100 samples per class).
for test_idx in selected_test_nodes:
    label_idx = graph.ndata['labels'][test_idx].item()
    class_name = gate_types[label_idx]
    
    lambda_max, x0, h_func = compute_hessian_for_sample(test_idx)
    class_hessian_eig[class_name].append(lambda_max)
    overall_hessian_eig.append(lambda_max)

# Display per-class results in a tabular format.
print("\nClass-wise Hessian-Based Curvature Results:")
header = "{:<10s} {:>35s}".format("Class", "Avg. Max Eigenvalue ± Std")
print(header)
print("-" * len(header))

for cls in gate_types:
    values = class_hessian_eig[cls]
    if len(values) > 0:
        avg_val = np.mean(values)
        std_val = np.std(values)
    else:
        avg_val, std_val = 0, 0
    print("{:<10s} {:>15.4f} ± {:<15.4f}".format(cls, avg_val, std_val))

# Compute and display the overall aggregated result.
overall_avg = np.mean(overall_hessian_eig)
overall_std = np.std(overall_hessian_eig)
print("\nOverall Aggregated Hessian-Based Curvature Measure:")
print("Average Max Eigenvalue: {:.4f} ± {:.4f}".format(overall_avg, overall_std))



--- Hessian-Based Curvature Measure Computation (100 Samples Per Class) ---


Class-wise Hessian-Based Curvature Results:
Class                Avg. Max Eigenvalue ± Std
----------------------------------------------
and                 0.0000 ± 0.0000         
or                  0.0000 ± 0.0000         
nand                0.0000 ± 0.0000         
nor                 0.0000 ± 0.0000         
xor                 0.0000 ± 0.0000         
xnor                0.0000 ± 0.0000         
buf                 0.0000 ± 0.0000         
not                 0.0000 ± 0.0000         

Overall Aggregated Hessian-Based Curvature Measure:
Average Max Eigenvalue: 0.0000 ± 0.0000


In [11]:
import torch
import numpy as np

print("\n--- Hessian-Based Curvature Measure & Relative Error Verification (100 Samples Per Class) ---\n")

# Use class names aligned with the label encoder to match label indices
class_names = list(label_encoder.classes_)

# -----------------------------
# 1) Build selected_test_nodes: up to 100 random test nodes per class
# -----------------------------
# Taken same as earlier metric (Local Lipschitz Constant)

# -----------------------------
# 2) Storage for metrics
# -----------------------------
class_hessian_eig = {cn: [] for cn in class_names}         # lambda_max (curvature proxy)
class_rel_errors  = {cn: [] for cn in class_names}         # relative error of second-order approximation
overall_hessian_eig = []
overall_rel_errors = []

# -----------------------------
# 3) Helper to compute H, grad, and scalar function h at x0
# -----------------------------
def compute_hessian_and_grad_for_sample(test_idx):
    """
    Compute Hessian H and gradient g of h(x) at x0 for node test_idx.
    h(x) = log-softmax(logits(x))[pred_class(x0)], i.e., log prob of class predicted at x0.
    """
    # Base input with gradients
    x0 = graph.ndata['features'][test_idx].clone().detach().requires_grad_(True)

    # Determine the predicted class at the unperturbed point
    with torch.no_grad():
        base_logits = model(graph, graph.ndata['features'])
        pred_class = torch.argmax(base_logits[test_idx])

    # Scalar function: log prob of the predicted class at x0
    def h(x):
        new_features = graph.ndata['features'].clone().detach()
        new_features[test_idx] = x
        logits = model(graph, new_features)[test_idx]
        log_probs = torch.log_softmax(logits, dim=0)
        return log_probs[pred_class]

    # Hessian (d^2 h / dx^2) at x0
    H = torch.autograd.functional.hessian(h, x0, create_graph=False, vectorize=False).detach()

    # Gradient g = dh/dx at x0
    x0_grad = torch.autograd.functional.jacobian(h, x0).detach()

    # Scalar baseline h(x0)
    h0 = h(x0.detach())

    return H, x0.detach(), x0_grad, h0, h

# -----------------------------
# 4) Main loop with second-order verification
# -----------------------------
epsilon = 5e-3          # small step; second-order term ~ O(eps^2)
trials_per_node = 10    # average over multiple directions to reduce noise

for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    # Compute Hessian, gradient, baseline
    H, x0, g, h0, h_func = compute_hessian_and_grad_for_sample(test_idx)

    # Curvature proxy: largest eigenvalue of H (symmetric)
    try:
        eigvals = torch.linalg.eigvalsh(H)
        lambda_max = torch.max(eigvals).item()
    except RuntimeError:
        # Fallback in rare numerical issues: use power iteration on symmetric H
        v = torch.randn_like(x0)
        v = v / (v.norm() + 1e-12)
        num_pi = 20
        for _ in range(num_pi):
            v = H @ v
            v_norm = v.norm() + 1e-12
            v = v / v_norm
        lambda_max = torch.dot(v, H @ v).item()

    # Relative error of second-order approximation:
    # predicted second-order change: 0.5 * delta^T H delta
    # actual second-order change: h(x0 + delta) - h(x0) - g^T delta
    node_errors = []
    for _ in range(trials_per_node):
        delta = epsilon * torch.randn_like(x0)

        # Predicted second-order change via Hessian quadratic form
        pred_second = 0.5 * torch.dot(delta, H @ delta).item()

        # Actual second-order change (subtract linear term)
        actual_second = (h_func(x0 + delta) - h0 - torch.dot(g, delta)).item()

        rel_error = abs(pred_second - actual_second) / (abs(actual_second) + 1e-8)
        node_errors.append(rel_error)

    avg_rel_error_node = float(np.mean(node_errors))

    # Store metrics
    class_hessian_eig[class_name].append(lambda_max)
    class_rel_errors[class_name].append(avg_rel_error_node)
    overall_hessian_eig.append(lambda_max)
    overall_rel_errors.append(avg_rel_error_node)

# -----------------------------
# 5) Reporting
# -----------------------------
print("\nClass-wise Hessian-Based Curvature (with Relative Errors):")
header = "{:<12s} {:>33s} {:>30s}".format("Class", "Avg. Max Eigenvalue ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    c_vals = class_hessian_eig[cn]
    e_vals = class_rel_errors[cn]
    if c_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(c_vals), np.std(c_vals), np.mean(e_vals), np.std(e_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:<10s} {:>15s} {:<10s}".format(cn, "-", "-", "-", "-"))

print("\nOverall Aggregated Hessian-Based Curvature:")
print("Avg Max Eigenvalue: {:.4f} ± {:.4f}".format(np.mean(overall_hessian_eig), np.std(overall_hessian_eig)))
print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))



--- Hessian-Based Curvature Measure & Relative Error Verification (100 Samples Per Class) ---


Class-wise Hessian-Based Curvature (with Relative Errors):
Class                Avg. Max Eigenvalue ± Std           Avg Rel. Error ± Std
-----------------------------------------------------------------------------
and               0.0000 ± 0.0000          1.6139e-01 ± 1.9902e-01
input             0.0000 ± 0.0000          9.4402e-01 ± 6.8588e-01
nand              0.0000 ± 0.0000          8.3175e-02 ± 1.3977e-01
nor               0.0000 ± 0.0000          1.0848e-01 ± 1.9403e-01
not               0.0000 ± 0.0000          1.7743e-01 ± 2.5927e-01
or                0.0000 ± 0.0000          7.6506e-01 ± 5.0202e+00
output            0.0000 ± 0.0000          6.1122e-01 ± 3.7771e-01
xor               0.0000 ± 0.0000          2.6412e-01 ± 2.0793e-01

Overall Aggregated Hessian-Based Curvature:
Avg Max Eigenvalue: 0.0000 ± 0.0000
Avg Relative Error: 4.0137e-01 ± 1.9147e+00


In [31]:
import numpy as np
import torch
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support

# -------------------------------------------------
# Assumes:
#  - `selected_test_nodes` already exists (from Hessian sampling step)
#  - `graph`, `model`, `label_encoder` already defined
#  - You may or may not have `jacobian_perturbations`; if absent, uses Gaussian noise
# -------------------------------------------------

# Ensure we have a NumPy array of node indices for the test set
if torch.is_tensor(test_mask):
    test_indices = test_mask.cpu().numpy()
else:
    test_indices = np.array(test_mask)

labels_np = graph.ndata['labels'].cpu().numpy()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)

rng = np.random.default_rng(seed=1)
PER_CLASS = 100
selected_test_nodes = []

print("\nSampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)

    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set — skipped.")
        continue

    if n_avail >= PER_CLASS:
        chosen = rng.choice(idxs, size=PER_CLASS, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = idxs
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")

    selected_test_nodes.extend(chosen)

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

labels_np = graph.ndata['labels'].cpu().numpy()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)

# Clone original features
perturbed_features = graph.ndata['features'].clone().detach()

epsilon = 0.05  # adjust perturbation strength if desired

# Apply perturbations to exactly the selected nodes
for node_idx in selected_test_nodes:
    x0 = graph.ndata['features'][node_idx]

    if 'jacobian_perturbations' in globals() and node_idx in jacobian_perturbations:
        delta = jacobian_perturbations[node_idx]
        norm = torch.norm(delta)
        if norm.item() > 0:
            delta = epsilon * delta / norm
    else:
        delta = epsilon * torch.randn_like(x0)

    perturbed_features[node_idx] = (x0 + delta)

# Evaluate only on perturbed subset
eval_idx = torch.tensor(selected_test_nodes, dtype=torch.long)

model.eval()
with torch.no_grad():
    logits_pert = model(graph, perturbed_features)
    preds = logits_pert[eval_idx].argmax(dim=1).cpu().numpy()
    labels_true = labels_np[eval_idx]

# ---- Metrics ----
acc = np.mean(preds == labels_true)
prec, rec, f1, _ = precision_recall_fscore_support(labels_true, preds, average='weighted', zero_division=0)

print(f"\n✅ Accuracy on Perturbed Set: {acc * 100:.2f}%")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")

# Confusion matrix across all classes
cm = confusion_matrix(labels_true, preds, labels=np.arange(num_classes))
print("\nConfusion Matrix (Perturbed Set, all classes):")
print(cm)

# Report only for present classes to avoid mismatch
present_labels = np.unique(labels_true)
present_names = [class_names[i] for i in present_labels]
print("\nClassification Report (present classes only):")
print(classification_report(labels_true, preds,
                            labels=present_labels,
                            target_names=present_names,
                            zero_division=0))



Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

✅ Accuracy on Perturbed Set: 68.22%
Precision: 0.7162
Recall:    0.6822
F1 Score:  0.6323

Confusion Matrix (Perturbed Set, all classes):
[[ 99   0   0   1   0   0   0   0]
 [  0 100   0   0   0   0   0   0]
 [ 82   0   7  11   0   0   0   0]
 [ 37   0   1  62   0   0   0   0]
 [  0   0   0   0 100   0   0   0]
 [ 99   0   1   0   0   0   0   0]
 [  0   0   0   0   0   0 100   0]
 [  0   0   0   0   0   0   0  30]]

Classification Report (present classes only):
              precision    recall  f1-score   support

         and       0.31      0.99      0.47       100
       input       1.00      1.00      1.00       100
        nand       0.78      0.07      0.

#### Prediction Margin

In [12]:
import torch
import numpy as np

print("\n--- Prediction Margin Computation (100 Samples Per Class) ---\n")

# Define gate types corresponding to your labels.
gate_types = ['and', 'or', 'nand', 'nor', 'xor', 'xnor', 'buf', 'not']

# Dictionary to store prediction margins for each gate type.
class_margin = {gt: [] for gt in gate_types}
overall_margin = []

# Iterate over each selected test node.
for test_idx in selected_test_nodes:
    # Obtain model output (logit vector) for the test sample.
    with torch.no_grad():
        logits = model(graph, graph.ndata['features'])[test_idx]  # f(x_i)
    
    # Predicted class: argmax_j f_j(x_i)
    pred_class = int(torch.argmax(logits))
    pred_logit = logits[pred_class].item()
    
    # Obtain the second highest value: set pred_class index to a very low number,
    # then take the max of the rest.
    other_logits = logits.clone()
    other_logits[pred_class] = -float('inf')
    second_max = other_logits.max().item()
    
    # Prediction margin: f_{y_pred}(x_i) - max_{j \neq y_pred} f_j(x_i)
    margin = pred_logit - second_max
    
    # Group the margin by the actual gate type label.
    label_idx = graph.ndata['labels'][test_idx].item()
    class_name = gate_types[label_idx]
    class_margin[class_name].append(margin)
    overall_margin.append(margin)

# Display per-class results in a tabular format.
print("\nClass-wise Prediction Margin Results:")
header = "{:<10s} {:>40s}".format("Class", "Avg. Margin ± Std")
print(header)
print("-" * len(header))
for cls in gate_types:
    values = class_margin[cls]
    if values:
        avg_val = np.mean(values)
        std_val = np.std(values)
    else:
        avg_val, std_val = 0, 0
    print("{:<10s} {:>15.4f} ± {:<15.4f}".format(cls, avg_val, std_val))

# Compute and display the overall aggregated prediction margin.
overall_avg_margin = np.mean(overall_margin)
overall_std_margin = np.std(overall_margin)
print("\nOverall Aggregated Prediction Margin:")
print("Average Margin: {:.4f} ± {:.4f}".format(overall_avg_margin, overall_std_margin))



--- Prediction Margin Computation (100 Samples Per Class) ---


Class-wise Prediction Margin Results:
Class                             Avg. Margin ± Std
---------------------------------------------------
and                 3.4587 ± 1.4896         
or                  5.7445 ± 0.8630         
nand                1.2225 ± 0.6649         
nor                 1.4505 ± 0.7046         
xor                 4.8346 ± 1.0507         
xnor                2.0727 ± 1.1462         
buf                 6.7341 ± 0.5587         
not                 4.5673 ± 0.4763         

Overall Aggregated Prediction Margin:
Average Margin: 3.6833 ± 2.2053


In [12]:
import torch
import numpy as np

print("\n--- Prediction Margin & Relative Error Verification (100 Samples Per Class) ---\n")

# -----------------------------
# 1) Use consistent class names from label encoder
# -----------------------------
class_names = list(label_encoder.classes_)

# Build selected_test_nodes: up to 100 random test nodes per class
rng = np.random.default_rng(42)
test_labels = graph.ndata['labels'][test_mask].cpu().numpy()
test_indices = np.where(test_mask.cpu().numpy())[0]

per_class_indices = {cls_idx: [] for cls_idx in range(len(class_names))}
for idx, lbl in zip(test_indices, test_labels):
    per_class_indices[lbl].append(idx)

selected_test_nodes = []
for cls_idx, idxs in per_class_indices.items():
    if idxs:
        k = min(100, len(idxs))
        chosen = rng.choice(idxs, size=k, replace=False).tolist()
        selected_test_nodes.extend(chosen)

# -----------------------------
# 2) Storage for metrics
# -----------------------------
class_margin_vals = {cn: [] for cn in class_names}
class_rel_errors  = {cn: [] for cn in class_names}
overall_margin_vals = []
overall_rel_errors = []

# -----------------------------
# 3) Main loop with relative error verification
# -----------------------------
epsilon = 1e-5   # perturbation for verification

for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    # Original logits
    with torch.no_grad():
        logits = model(graph, graph.ndata['features'])[test_idx]

    pred_class = int(torch.argmax(logits))
    pred_logit = logits[pred_class].item()

    # Mask out predicted class to find second max
    other_logits = logits.clone()
    other_logits[pred_class] = -float('inf')
    second_max = other_logits.max().item()

    margin = pred_logit - second_max

    # -----------------------------
    # Relative error verification:
    # Compare to recomputation after a small perturbation in features
    # -----------------------------
    # Perturb node features slightly
    perturbed_feats = graph.ndata['features'].clone()
    perturb_vec = torch.randn_like(perturbed_feats[test_idx]) * epsilon
    perturbed_feats[test_idx] += perturb_vec

    with torch.no_grad():
        logits_pert = model(graph, perturbed_feats)[test_idx]
    pred_logit_pert = logits_pert[pred_class].item()
    other_logits_pert = logits_pert.clone()
    other_logits_pert[pred_class] = -float('inf')
    second_max_pert = other_logits_pert.max().item()
    margin_pert = pred_logit_pert - second_max_pert

    rel_err = abs(margin - margin_pert) / (abs(margin_pert) + 1e-12)

    # Store
    class_margin_vals[class_name].append(margin)
    class_rel_errors[class_name].append(rel_err)
    overall_margin_vals.append(margin)
    overall_rel_errors.append(rel_err)

# -----------------------------
# 4) Reporting
# -----------------------------
print("\nClass-wise Prediction Margin (with Relative Error):")
header = "{:<12s} {:>33s} {:>30s}".format("Class", "Avg. Margin ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    m_vals = class_margin_vals[cn]
    e_vals = class_rel_errors[cn]
    if m_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(m_vals), np.std(m_vals),
            np.mean(e_vals), np.std(e_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:<10s} {:>15s} {:<10s}".format(cn, "-", "-", "-", "-"))

print("\nOverall Aggregated Prediction Margin:")
print("Avg Margin: {:.4f} ± {:.4f}".format(np.mean(overall_margin_vals), np.std(overall_margin_vals)))
print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))



--- Prediction Margin & Relative Error Verification (100 Samples Per Class) ---


Class-wise Prediction Margin (with Relative Error):
Class                        Avg. Margin ± Std           Avg Rel. Error ± Std
-----------------------------------------------------------------------------
and               4.1818 ± 1.0851          9.6553e-06 ± 7.8073e-06
input             6.8378 ± 0.0000          1.1052e-06 ± 9.5447e-07
nand                   - -                        - -         
nor                    - -                        - -         
not               4.9227 ± 0.6428          9.6125e-06 ± 7.7242e-06
or                     - -                        - -         
output            6.4653 ± 0.0005          1.6930e-06 ± 1.4186e-06
xor                    - -                        - -         

Overall Aggregated Prediction Margin:
Avg Margin: 4.6119 ± 1.0198
Avg Relative Error: 9.3943e-06 ± 7.7819e-06


In [32]:
import numpy as np
import torch
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support

# -------------------------------------------------
# Assumes:
#  - `selected_test_nodes` already exists (from Prediction Margin sampling step)
#  - `graph`, `model`, `label_encoder` already defined
#  - Optionally: `jacobian_perturbations` dict for node_idx -> perturbation tensor
# -------------------------------------------------

# Ensure we have a NumPy array of node indices for the test set
if torch.is_tensor(test_mask):
    test_indices = test_mask.cpu().numpy()
else:
    test_indices = np.array(test_mask)

labels_np = graph.ndata['labels'].cpu().numpy()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)

rng = np.random.default_rng(seed=22)
PER_CLASS = 100
selected_test_nodes = []

print("\nSampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)

    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set — skipped.")
        continue

    if n_avail >= PER_CLASS:
        chosen = rng.choice(idxs, size=PER_CLASS, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = idxs
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")

    selected_test_nodes.extend(chosen)

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

labels_np = graph.ndata['labels'].cpu().numpy()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)

# Clone original features
perturbed_features = graph.ndata['features'].clone().detach()

epsilon = 0.05  # adjust perturbation strength if desired

# Apply perturbations to exactly the selected nodes
for node_idx in selected_test_nodes:
    x0 = graph.ndata['features'][node_idx]

    if 'jacobian_perturbations' in globals() and node_idx in jacobian_perturbations:
        delta = jacobian_perturbations[node_idx]
        norm = torch.norm(delta)
        if norm.item() > 0:
            delta = epsilon * delta / norm
    else:
        delta = epsilon * torch.randn_like(x0)

    perturbed_features[node_idx] = (x0 + delta)

# Evaluate only on perturbed subset
eval_idx = torch.tensor(selected_test_nodes, dtype=torch.long)

model.eval()
with torch.no_grad():
    logits_pert = model(graph, perturbed_features)
    preds = logits_pert[eval_idx].argmax(dim=1).cpu().numpy()
    labels_true = labels_np[eval_idx]

# ---- Metrics ----
acc = np.mean(preds == labels_true)
prec, rec, f1, _ = precision_recall_fscore_support(labels_true, preds, average='weighted', zero_division=0)

print(f"\n✅ Accuracy on Perturbed Set: {acc * 100:.2f}%")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")

# Confusion matrix across all classes
cm = confusion_matrix(labels_true, preds, labels=np.arange(num_classes))
print("\nConfusion Matrix (Perturbed Set, all classes):")
print(cm)

# Report only for present classes to avoid mismatch
present_labels = np.unique(labels_true)
present_names = [class_names[i] for i in present_labels]
print("\nClassification Report (present classes only):")
print(classification_report(labels_true, preds,
                            labels=present_labels,
                            target_names=present_names,
                            zero_division=0))



Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

✅ Accuracy on Perturbed Set: 70.27%
Precision: 0.7263
Recall:    0.7027
F1 Score:  0.6508

Confusion Matrix (Perturbed Set, all classes):
[[ 99   0   0   1   0   0   0   0]
 [  0  99   1   0   0   0   0   0]
 [ 82   0   8  10   0   0   0   0]
 [ 22   0   1  77   0   0   0   0]
 [  0   0   0   0 100   0   0   0]
 [100   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0 100   0]
 [  0   0   0   0   0   0   0  30]]

Classification Report (present classes only):
              precision    recall  f1-score   support

         and       0.33      0.99      0.49       100
       input       1.00      0.99      0.99       100
        nand       0.80      0.08      0.

#### Adversarial Robustness Radius

In [13]:
import torch
import numpy as np

print("\n--- Adversarial Robustness Radius Computation (100 Samples Per Class) ---\n")

# Dictionary to store the adversarial robustness radius for each gate type.
class_adv_radius = {gt: [] for gt in gate_types}
overall_adv_radius = []

def f_for_sample(x, test_idx):
    """
    Given an input x (modified feature vector for the test node),
    returns the output (logit vector) for that node.
    """
    new_features = graph.ndata['features'].clone().detach()
    new_features[test_idx] = x
    with torch.no_grad():
        out = model(graph, new_features)
    return out[test_idx]

def adversarial_radius_for_sample(test_idx, initial_epsilon=1e-3, growth_factor=1.2, max_epsilon=10.0, bs_iters=10, num_trials=10):
    """
    For a given test node (identified by test_idx), this function computes the smallest
    perturbation norm (adversarial robustness radius) required to change the model's prediction.
    We sample a number of random directions and, for each, increase the perturbation until
    the predicted class changes; a binary search is then performed to refine the minimal epsilon.
    The minimal epsilon over all trials is returned.
    """
    # Retrieve the original feature vector.
    x0 = graph.ndata['features'][test_idx].clone().detach()
    
    # Determine the predicted class at x0.
    with torch.no_grad():
        out = model(graph, graph.ndata['features'])
        y0 = torch.argmax(out[test_idx]).item()
        
    # Helper function: returns True if prediction at x remains unchanged.
    def is_same(x):
        out = f_for_sample(x, test_idx)
        return (torch.argmax(out).item() == y0)
    
    radii = []
    for _ in range(num_trials):
        # Sample a random direction.
        d = torch.randn_like(x0)
        d = d / (torch.norm(d) + 1e-8)
        epsilon = initial_epsilon
        
        # Increase epsilon until the classification changes or max_epsilon is reached.
        while epsilon < max_epsilon and is_same(x0 + epsilon * d):
            epsilon *= growth_factor
        
        # If maximum epsilon is reached, use max_epsilon as candidate.
        if epsilon >= max_epsilon:
            candidate = max_epsilon
        else:
            # Refine the candidate using binary search between [epsilon/growth_factor, epsilon].
            low = epsilon / growth_factor
            high = epsilon
            for _ in range(bs_iters):
                mid = (low + high) / 2
                if is_same(x0 + mid * d):
                    low = mid
                else:
                    high = mid
            candidate = high
        radii.append(candidate)
    
    return min(radii)

# Iterate over each selected test node (assumed 100 samples per class are in selected_test_nodes).
for test_idx in selected_test_nodes:
    label_idx = graph.ndata['labels'][test_idx].item()
    class_name = gate_types[label_idx]
    
    radius = adversarial_radius_for_sample(test_idx)
    class_adv_radius[class_name].append(radius)
    overall_adv_radius.append(radius)

# Display per-class results.
print("\nClass-wise Adversarial Robustness Radius Results:")
header = "{:<10s} {:>35s}".format("Class", "Avg. Radius ± Std")
print(header)
print("-" * len(header))
for cls in gate_types:
    values = class_adv_radius[cls]
    if values:
        avg_val = np.mean(values)
        std_val = np.std(values)
    else:
        avg_val, std_val = 0, 0
    print("{:<10s} {:>15.4f} ± {:<15.4f}".format(cls, avg_val, std_val))
    
# Compute overall aggregated results.
overall_avg = np.mean(overall_adv_radius)
overall_std = np.std(overall_adv_radius)
print("\nOverall Aggregated Adversarial Robustness Radius:")
print("Average Radius: {:.4f} ± {:.4f}".format(overall_avg, overall_std))



--- Adversarial Robustness Radius Computation (100 Samples Per Class) ---



KeyboardInterrupt: 

In [14]:
import torch
import numpy as np

print("\n--- Adversarial Robustness Radius & Relative Error Verification (100 Samples Per Class) ---\n")

# -----------------------------
# 1) Use consistent class names from label encoder and sample nodes
# -----------------------------
class_names = list(label_encoder.classes_)

rng = np.random.default_rng(42)
test_labels = graph.ndata['labels'][test_mask].cpu().numpy()
test_indices = np.where(test_mask.cpu().numpy())[0]

per_class_indices = {cls_idx: [] for cls_idx in range(len(class_names))}
for idx, lbl in zip(test_indices, test_labels):
    per_class_indices[lbl].append(idx)

selected_test_nodes = []
for cls_idx, idxs in per_class_indices.items():
    if idxs:
        k = min(100, len(idxs))
        chosen = rng.choice(idxs, size=k, replace=False).tolist()
        selected_test_nodes.extend(chosen)

# -----------------------------
# 2) Storage
# -----------------------------
class_adv_radius = {cn: [] for cn in class_names}
class_rel_errors  = {cn: [] for cn in class_names}
overall_adv_radius_vals = []
overall_rel_errors      = []

# -----------------------------
# 3) Prediction wrapper for a node
# -----------------------------
def f_for_sample(x, test_idx):
    """
    Given an input x (modified feature vector for the test node),
    returns the output (logit vector) for that node.
    """
    new_features = graph.ndata['features'].clone().detach()
    new_features[test_idx] = x
    with torch.no_grad():
        out = model(graph, new_features)
    return out[test_idx]

# -----------------------------
# 4) Adversarial radius along random directions + binary search
# -----------------------------
def adversarial_radius_for_sample(test_idx, initial_epsilon=1e-3, growth_factor=1.2,
                                  max_epsilon=10.0, bs_iters=10, num_trials=10):
    """
    Compute the minimal perturbation norm required to change the prediction,
    by sampling random directions d (||d||=1), expanding epsilon until flip,
    then binary-searching within the bracket. Returns the minimum over trials.
    """
    x0 = graph.ndata['features'][test_idx].clone().detach()

    with torch.no_grad():
        base_out = model(graph, graph.ndata['features'])
        y0 = int(torch.argmax(base_out[test_idx]).item())

    def is_same(x):
        out = f_for_sample(x, test_idx)
        return int(torch.argmax(out).item()) == y0

    radii = []
    for _ in range(num_trials):
        d = torch.randn_like(x0)
        d = d / (torch.norm(d) + 1e-12)

        epsilon = initial_epsilon
        # Expand until flip or cap
        while epsilon < max_epsilon and is_same(x0 + epsilon * d):
            epsilon *= growth_factor

        if epsilon >= max_epsilon:
            candidate = max_epsilon
        else:
            # Binary search in [epsilon/growth_factor, epsilon]
            low = epsilon / growth_factor
            high = epsilon
            for _ in range(bs_iters):
                mid = 0.5 * (low + high)
                if is_same(x0 + mid * d):
                    low = mid
                else:
                    high = mid
            candidate = high
        radii.append(candidate)

    return float(min(radii))

# -----------------------------
# 5) Relative error verification
# -----------------------------
# We verify by re-estimating the radius with a different randomness seed/config
# and measuring |r - r'| / (r' + 1e-12). This checks stability of the estimator.
def adversarial_radius_relerr(test_idx):
    r1 = adversarial_radius_for_sample(test_idx, initial_epsilon=1e-3, growth_factor=1.2,
                                       max_epsilon=10.0, bs_iters=10, num_trials=10)
    r2 = adversarial_radius_for_sample(test_idx, initial_epsilon=1e-3, growth_factor=1.3,
                                       max_epsilon=10.0, bs_iters=12, num_trials=10)
    rel_err = abs(r1 - r2) / (abs(r2) + 1e-12)
    return r1, rel_err

# -----------------------------
# 6) Main loop
# -----------------------------
for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    radius, rel_err = adversarial_radius_relerr(test_idx)

    class_adv_radius[class_name].append(radius)
    class_rel_errors[class_name].append(rel_err)
    overall_adv_radius_vals.append(radius)
    overall_rel_errors.append(rel_err)

# -----------------------------
# 7) Reporting
# -----------------------------
print("\nClass-wise Adversarial Robustness Radius (with Relative Error):")
header = "{:<12s} {:>27s} {:>30s}".format("Class", "Avg. Radius ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    r_vals = class_adv_radius[cn]
    e_vals = class_rel_errors[cn]
    if r_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(r_vals), np.std(r_vals), np.mean(e_vals), np.std(e_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:<10s} {:>15s} {:<10s}".format(cn, "-", "-", "-", "-"))

print("\nOverall Aggregated Adversarial Robustness Radius:")
print("Avg Radius: {:.4f} ± {:.4f}".format(np.mean(overall_adv_radius_vals), np.std(overall_adv_radius_vals)))
print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))



--- Adversarial Robustness Radius & Relative Error Verification (100 Samples Per Class) ---



KeyboardInterrupt: 

In [33]:
import numpy as np
import torch
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support

# -------------------------------------------------
# Assumes:
#  - `selected_test_nodes` already exists (from ARR sampling step)
#  - `graph`, `model`, `label_encoder` already defined
#  - Optionally: `jacobian_perturbations` dict for node_idx -> perturbation tensor
# -------------------------------------------------

# Ensure we have a NumPy array of node indices for the test set
if torch.is_tensor(test_mask):
    test_indices = test_mask.cpu().numpy()
else:
    test_indices = np.array(test_mask)

labels_np = graph.ndata['labels'].cpu().numpy()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)

rng = np.random.default_rng(seed=18)
PER_CLASS = 100
selected_test_nodes = []

print("\nSampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)

    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set — skipped.")
        continue

    if n_avail >= PER_CLASS:
        chosen = rng.choice(idxs, size=PER_CLASS, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = idxs
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")

    selected_test_nodes.extend(chosen)

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

labels_np = graph.ndata['labels'].cpu().numpy()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)

# Clone original features
perturbed_features = graph.ndata['features'].clone().detach()

epsilon = 0.05  # adjust perturbation strength if desired

# Apply perturbations to exactly the selected nodes
for node_idx in selected_test_nodes:
    x0 = graph.ndata['features'][node_idx]

    if 'jacobian_perturbations' in globals() and node_idx in jacobian_perturbations:
        delta = jacobian_perturbations[node_idx]
        norm = torch.norm(delta)
        if norm.item() > 0:
            delta = epsilon * delta / norm
    else:
        delta = epsilon * torch.randn_like(x0)

    perturbed_features[node_idx] = (x0 + delta)

# Evaluate only on perturbed subset
eval_idx = torch.tensor(selected_test_nodes, dtype=torch.long)

model.eval()
with torch.no_grad():
    logits_pert = model(graph, perturbed_features)
    preds = logits_pert[eval_idx].argmax(dim=1).cpu().numpy()
    labels_true = labels_np[eval_idx]

# ---- Metrics ----
acc = np.mean(preds == labels_true)
prec, rec, f1, _ = precision_recall_fscore_support(labels_true, preds, average='weighted', zero_division=0)

print(f"\n✅ Accuracy on Perturbed Set: {acc * 100:.2f}%")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")

# Confusion matrix across all classes
cm = confusion_matrix(labels_true, preds, labels=np.arange(num_classes))
print("\nConfusion Matrix (Perturbed Set, all classes):")
print(cm)

# Report only for present classes to avoid mismatch
present_labels = np.unique(labels_true)
present_names = [class_names[i] for i in present_labels]
print("\nClassification Report (present classes only):")
print(classification_report(labels_true, preds,
                            labels=present_labels,
                            target_names=present_names,
                            zero_division=0))



Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

✅ Accuracy on Perturbed Set: 70.14%
Precision: 0.6945
Recall:    0.7014
F1 Score:  0.6452

Confusion Matrix (Perturbed Set, all classes):
[[100   0   0   0   0   0   0   0]
 [  0 100   0   0   0   0   0   0]
 [ 85   0   5  10   0   0   0   0]
 [ 22   0   1  77   0   0   0   0]
 [  0   0   0   0 100   0   0   0]
 [ 97   0   3   0   0   0   0   0]
 [  0   0   0   0   0   0 100   0]
 [  0   0   0   0   0   0   0  30]]

Classification Report (present classes only):
              precision    recall  f1-score   support

         and       0.33      1.00      0.50       100
       input       1.00      1.00      1.00       100
        nand       0.56      0.05      0.

#### Stability Under Input Noise

In [14]:
import torch
import numpy as np

print("\n--- Stability Under Input Noise Computation (100 Samples Per Class) ---\n")

# Standard deviation for Gaussian noise perturbations.
sigma = 0.01

# Number of noise samples to approximate the expectation.
num_noise_samples = 20

# Define gate types corresponding to your labels.
gate_types = ['and', 'or', 'nand', 'nor', 'xor', 'xnor', 'buf', 'not']

# Dictionaries to store stability metric per gate type and overall.
class_stability = {gt: [] for gt in gate_types}
overall_stability = []

def stability_for_sample(test_idx, sigma, num_samples):
    """
    Compute the stability under input noise for a given test sample.
    For the sample at test_idx, the prediction difference is computed as the Euclidean norm 
    between the model outputs for the original and the noisy inputs. The stability measure 
    S(x_i) is obtained by averaging over multiple noise samples.
    """
    # Retrieve the original feature vector.
    x0 = graph.ndata['features'][test_idx].clone().detach()
    
    # Compute the model output for the unperturbed input.
    with torch.no_grad():
        f_orig = model(graph, graph.ndata['features'])[test_idx]
    
    differences = []
    for _ in range(num_samples):
        # Sample noise from a Gaussian distribution: N(0, sigma^2 I)
        noise = sigma * torch.randn_like(x0)
        x_noisy = x0 + noise
        
        # Replace the test node's feature with the noisy feature.
        new_features = graph.ndata['features'].clone().detach()
        new_features[test_idx] = x_noisy
        
        # Compute the model's output for the noisy input.
        with torch.no_grad():
            f_noisy = model(graph, new_features)[test_idx]
        
        # Calculate the Euclidean norm difference between the noisy and original outputs.
        diff = torch.norm(f_noisy - f_orig).item()
        differences.append(diff)
    
    # Average difference approximates the expectation.
    return np.mean(differences)

# Iterate over each selected test node (assumed to be 100 samples per class in selected_test_nodes).
for test_idx in selected_test_nodes:
    label_idx = graph.ndata['labels'][test_idx].item()
    class_name = gate_types[label_idx]
    
    stability_measure = stability_for_sample(test_idx, sigma, num_noise_samples)
    class_stability[class_name].append(stability_measure)
    overall_stability.append(stability_measure)

# Display per-class results.
print("\nClass-wise Stability Under Input Noise Results:")
header = "{:<10s} {:>40s}".format("Class", "Avg. Stability (S(x)) ± Std")
print(header)
print("-" * len(header))
for cls in gate_types:
    values = class_stability[cls]
    if values:
        avg_val = np.mean(values)
        std_val = np.std(values)
    else:
        avg_val, std_val = 0, 0
    print("{:<10s} {:>15.4f} ± {:<15.4f}".format(cls, avg_val, std_val))
    
overall_avg = np.mean(overall_stability)
overall_std = np.std(overall_stability)
print("\nOverall Aggregated Stability Under Input Noise:")
print("Average Stability: {:.4f} ± {:.4f}".format(overall_avg, overall_std))



--- Stability Under Input Noise Computation (100 Samples Per Class) ---


Class-wise Stability Under Input Noise Results:
Class                   Avg. Stability (S(x)) ± Std
---------------------------------------------------
and                 0.0692 ± 0.0177         
or                  0.0458 ± 0.0049         
nand                0.0597 ± 0.0185         
nor                 0.0458 ± 0.0103         
xor                 0.0810 ± 0.0126         
xnor                0.0535 ± 0.0224         
buf                 0.0383 ± 0.0040         
not                 0.0270 ± 0.0030         

Overall Aggregated Stability Under Input Noise:
Average Stability: 0.0550 ± 0.0204


In [13]:
import torch
import numpy as np

print("\n--- Stability Under Input Noise & Relative Error Verification (100 Samples Per Class) ---\n")

# -----------------------------
# 1) Use consistent class names from label encoder
# -----------------------------
class_names = list(label_encoder.classes_)

# Build selected_test_nodes: up to 100 random test nodes per class
rng = np.random.default_rng(42)
test_labels = graph.ndata['labels'][test_mask].cpu().numpy()
test_indices = np.where(test_mask.cpu().numpy())[0]

per_class_indices = {cls_idx: [] for cls_idx in range(len(class_names))}
for idx, lbl in zip(test_indices, test_labels):
    per_class_indices[lbl].append(idx)

selected_test_nodes = []
for cls_idx, idxs in per_class_indices.items():
    if idxs:
        k = min(100, len(idxs))
        chosen = rng.choice(idxs, size=k, replace=False).tolist()
        selected_test_nodes.extend(chosen)

# -----------------------------
# 2) Parameters
# -----------------------------
sigma = 0.01          # stddev of Gaussian noise
num_noise_samples = 20
relerr_resamples  = 5 # verification repetitions

# -----------------------------
# 3) Storage
# -----------------------------
class_stability   = {cn: [] for cn in class_names}
class_rel_errors  = {cn: [] for cn in class_names}
overall_stability_vals = []
overall_rel_errors     = []

# -----------------------------
# 4) Helpers
# -----------------------------
def stability_for_sample(test_idx, sigma, num_samples):
    """
    Compute stability under Gaussian noise: average L2 change in logits
    between clean and noisy versions of the given test node.
    """
    x0 = graph.ndata['features'][test_idx].clone().detach()

    with torch.no_grad():
        f_orig = model(graph, graph.ndata['features'])[test_idx]

    diffs = []
    for _ in range(num_samples):
        noise = sigma * torch.randn_like(x0)
        x_noisy = x0 + noise

        new_feats = graph.ndata['features'].clone().detach()
        new_feats[test_idx] = x_noisy

        with torch.no_grad():
            f_noisy = model(graph, new_feats)[test_idx]

        diffs.append(torch.norm(f_noisy - f_orig).item())

    return float(np.mean(diffs))

# -----------------------------
# 5) Main loop
# -----------------------------
for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    stab_val = stability_for_sample(test_idx, sigma, num_noise_samples)

    # Relative error check: re‑estimate stability with fresh noise and compare
    re_vals = [stability_for_sample(test_idx, sigma, num_noise_samples)
               for _ in range(relerr_resamples)]
    avg_reval = float(np.mean(re_vals))
    rel_err = abs(stab_val - avg_reval) / (abs(avg_reval) + 1e-12)

    class_stability[class_name].append(stab_val)
    class_rel_errors[class_name].append(rel_err)
    overall_stability_vals.append(stab_val)
    overall_rel_errors.append(rel_err)

# -----------------------------
# 6) Reporting
# -----------------------------
print("\nClass-wise Stability Under Input Noise (with Relative Error):")
header = "{:<12s} {:>33s} {:>30s}".format("Class", "Avg. Stability ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    s_vals = class_stability[cn]
    e_vals = class_rel_errors[cn]
    if s_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(s_vals), np.std(s_vals),
            np.mean(e_vals), np.std(e_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:<10s} {:>15s} {:<10s}".format(cn, "-", "-", "-", "-"))

print("\nOverall Aggregated Stability Under Input Noise:")
print("Avg Stability: {:.4f} ± {:.4f}".format(np.mean(overall_stability_vals), np.std(overall_stability_vals)))
print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))



--- Stability Under Input Noise & Relative Error Verification (100 Samples Per Class) ---


Class-wise Stability Under Input Noise (with Relative Error):
Class                     Avg. Stability ± Std           Avg Rel. Error ± Std
-----------------------------------------------------------------------------
and               0.0724 ± 0.0122          9.1036e-02 ± 6.9657e-02
input             0.0531 ± 0.0074          1.3913e-01 ± 1.1634e-01
nand                   - -                        - -         
nor                    - -                        - -         
not               0.0790 ± 0.0106          8.5734e-02 ± 5.9886e-02
or                     - -                        - -         
output            0.0440 ± 0.0045          8.4250e-02 ± 6.5533e-02
xor                    - -                        - -         

Overall Aggregated Stability Under Input Noise:
Avg Stability: 0.0749 ± 0.0125
Avg Relative Error: 8.9337e-02 ± 6.6771e-02


In [34]:
import numpy as np
import torch
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support

# -------------------------------------------------
# Assumes:
#  - `selected_test_nodes` already exists (from Stability sampling step)
#  - `graph`, `model`, `label_encoder` already defined
#  - Optionally: `jacobian_perturbations` dict for node_idx -> perturbation tensor
# -------------------------------------------------

# Ensure we have a NumPy array of node indices for the test set
if torch.is_tensor(test_mask):
    test_indices = test_mask.cpu().numpy()
else:
    test_indices = np.array(test_mask)

labels_np = graph.ndata['labels'].cpu().numpy()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)

rng = np.random.default_rng(seed=50)
PER_CLASS = 100
selected_test_nodes = []

print("\nSampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)

    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set — skipped.")
        continue

    if n_avail >= PER_CLASS:
        chosen = rng.choice(idxs, size=PER_CLASS, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = idxs
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")

    selected_test_nodes.extend(chosen)

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

labels_np = graph.ndata['labels'].cpu().numpy()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)

# Clone original features
perturbed_features = graph.ndata['features'].clone().detach()

epsilon = 0.05  # adjust perturbation strength if desired

# Apply perturbations to exactly the selected nodes
for node_idx in selected_test_nodes:
    x0 = graph.ndata['features'][node_idx]

    if 'jacobian_perturbations' in globals() and node_idx in jacobian_perturbations:
        delta = jacobian_perturbations[node_idx]
        norm = torch.norm(delta)
        if norm.item() > 0:
            delta = epsilon * delta / norm
    else:
        delta = epsilon * torch.randn_like(x0)

    perturbed_features[node_idx] = (x0 + delta)

# Evaluate only on perturbed subset
eval_idx = torch.tensor(selected_test_nodes, dtype=torch.long)

model.eval()
with torch.no_grad():
    logits_pert = model(graph, perturbed_features)
    preds = logits_pert[eval_idx].argmax(dim=1).cpu().numpy()
    labels_true = labels_np[eval_idx]

# ---- Metrics ----
acc = np.mean(preds == labels_true)
prec, rec, f1, _ = precision_recall_fscore_support(labels_true, preds, average='weighted', zero_division=0)

print(f"\n✅ Accuracy on Perturbed Set: {acc * 100:.2f}%")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")

# Confusion matrix across all classes
cm = confusion_matrix(labels_true, preds, labels=np.arange(num_classes))
print("\nConfusion Matrix (Perturbed Set, all classes):")
print(cm)

# Report only for present classes to avoid mismatch
present_labels = np.unique(labels_true)
present_names = [class_names[i] for i in present_labels]
print("\nClassification Report (present classes only):")
print(classification_report(labels_true, preds,
                            labels=present_labels,
                            target_names=present_names,
                            zero_division=0))



Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

✅ Accuracy on Perturbed Set: 69.59%
Precision: 0.7203
Recall:    0.6959
F1 Score:  0.6431

Confusion Matrix (Perturbed Set, all classes):
[[100   0   0   0   0   0   0   0]
 [  0 100   0   0   0   0   0   0]
 [ 81   0   7  12   0   0   0   0]
 [ 29   0   0  71   0   0   0   0]
 [  0   0   0   0 100   0   0   0]
 [ 98   0   2   0   0   0   0   0]
 [  0   0   0   0   0   0 100   0]
 [  0   0   0   0   0   0   0  30]]

Classification Report (present classes only):
              precision    recall  f1-score   support

         and       0.32      1.00      0.49       100
       input       1.00      1.00      1.00       100
        nand       0.78      0.07      0.